In [21]:
from google.cloud import storage
import datetime
import pandas as pd
from io import BytesIO
import os

In [31]:
client = storage.Client()
bucket = client.get_bucket('royale-data')
blobs = bucket.list_blobs(prefix='user_data/')
blob_names = [b.name for b in blobs]
num_users = len(blob_names) - 1
date = (datetime.datetime.utcnow() - datetime.timedelta(days=1)).strftime('%m/%d/%Y')

df = pd.read_csv(BytesIO(bucket.get_blob('data/daily_user_count.csv').download_as_string()))
df.append(pd.DataFrame({'date':[date], 'count':[num_users]}))
df = df.groupby('date', as_index=False).first()

df

,date,count
0,05/26/2020,6


In [ ]:
# save to storage
fname = 'daily_user_count.csv'
local_fname = '/tmp/'+fname
df.to_csv(local_fname, index=False)
BUCKET.blob('data/{}'.format(fname)).upload_from_file(open(local_fname, 'rb'))
os.remove(local_fname)